In [ ]:
#|default_exp core

# Claudia

## Setup

- basic and streaming chat
- images
- tool use

In [ ]:
import os
# os.environ['ANTHROPIC_LOG'] = 'debug'

In [ ]:
#| export
import xml.etree.ElementTree as ET, json

from anthropic import Anthropic
from fastcore.utils import *

In [ ]:
#| export
models = 'claude-3-opus-20240229','claude-3-sonnet-20240229','claude-3-haiku-20240307'

In [ ]:
model = models[1]

## Helpers

In [ ]:
#| export
def mk_msg(content, role='user', **kw):
    "Helper to create a `dict` appropriate for a Claude message"
    return dict(role=role, content=content, **kw)

In [ ]:
#| export
def mk_msgs(msgs, **kw):
    "Helper to set 'assistant' role on alternate messages"
    return [mk_msg(o, ('user','assistant')[i%2], **kw) if isinstance(o,str) else o
            for i,o in enumerate(msgs)]

In [ ]:
#| export
def contents(r):
    "Help to get the contents from Claude response `r`"
    return r.content[0].text.strip()

In [ ]:
#| export
class AnthClient:
    def __init__(self, model, cli=None):
        "Basic Anthropic messages client"
        self.m,self.model = (cli or Anthropic()).messages,model

    def __call__(self, msgs, sp='', temp=0, maxtok=4096, stop=None):
        return self.m.create(model=self.model, messages=mk_msgs(msgs), max_tokens=maxtok,
                             system=sp, temperature=temp, stop_sequences=stop)

    def stream(self, msgs, sp='', temp=0, maxtok=4096, stop=None):
        with self.m.stream(model=self.model, messages=mk_msgs(msgs), max_tokens=maxtok,
            system=sp, temperature=temp, stop_sequences=stop) as stream:
            yield from stream.text_stream

In [ ]:
c = AnthClient(models[-1])

In [ ]:
#| export
def to_xml(node, hl=False):
    "Convert `node` to an XML string"
    def mk_el(tag, cs, attrs):
        el = ET.Element(tag, attrib=attrs)
        if isinstance(cs, list): el.extend([mk_el(*o) for o in cs])
        elif cs is not None: el.text = str(cs)
        return el

    root = mk_el(*node)
    ET.indent(root)
    res = ET.tostring(root, encoding='unicode')
    return hl_md(res) if hl else res

In [ ]:
#| export
def xt(tag, c=None, **kw):
    "Helper to create appropriate data structure for `to_xml`"
    kw = {k.lstrip('_'):str(v) for k,v in kw.items()}
    return tag,c,kw

In [ ]:
#| export
g = globals()
tags = 'div','img','h1','h2','h3','h4','h5','p','hr','span','html'
for o in tags: g[o] = partial(t, o)

In [ ]:
a = html([
    p('This is a paragraph'),
    hr(),
    xt('x-custom', foo='bar'),
    img(src='http://example.prg'),
    div([
        h1('This is a header'),
        h2('This is a sub-header', style='k:v'),
    ], _class='foo')
])

In [ ]:
to_xml(a, True)

```xml
<html>
  <p>This is a paragraph</p>
  <hr />
  <x-custom foo="bar" />
  <img src="http://example.prg" />
  <div class="foo">
    <h1>This is a header</h1>
    <h2 style="k:v">This is a sub-header</h2>
  </div>
</html>
```

In [ ]:
#|export
def json_to_xml(d, rnm):
    root = ET.Element(rnm)
    def build_xml(data, parent):
        if isinstance(data, dict):
            for key, value in data.items(): build_xml(value, ET.SubElement(parent, key))
        elif isinstance(data, list):
            for item in data: build_xml(item, ET.SubElement(parent, 'item'))
        else: parent.text = str(data)
    build_xml(d, root)
    ET.indent(root)
    return ET.tostring(root, encoding='unicode')

In [ ]:
#| export
def first_match(lst, f, default=None):
    "First element of `lst` matching predicate `f`, or `default` if none"
    return next((i for i,o in enumerate(lst) if f(o)), default)

#| export
def last_match(lst, f, default=None):
    "Last element of `lst` matching predicate `f`, or `default` if none"
    return next((i for i in range(len(lst)-1, -1, -1) if f(lst[i])), default)

## Export -

In [ ]:
#|hide
#|eval: false
from nbdev.doclinks import nbdev_export
nbdev_export()